In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from config import gkey, geoapify_key, census_key

census_key

/Users/glena.dagger/Downloads/anaconda3/envs/pythondata/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


ImportError: cannot import name 'census_key'